In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ozelgun = pd.read_csv('Dim_OzelGunTanimlari.csv')
imha = pd.read_csv('imha_2014-2016.txt', sep = '\t')
satis = pd.read_csv('satis_2014-2016.txt', sep = '\t')
stok = pd.read_csv('stok_2014-2016.txt', sep = '\t')
mal = pd.read_csv('mal_2016.csv', sep = ';') 

In [3]:
satis['TarihIndex'] = pd.to_datetime(satis['Tarih'], format='%d.%m.%Y')
imha['TarihIndex'] = pd.to_datetime(imha['Tarih'], format='%d.%m.%Y')
stok['TarihIndex'] = pd.to_datetime(stok['Tarih'], format= '%d.%m.%Y')

In [4]:
mal['KategoriTamami'] = mal["KategoriAnaGrupNo"].map(
    str)+ "" + mal["KategoriGrupNo"].map(str)+ "" + mal["KategoriSinifNo"].map(
    str)+ "" + mal["KategoriAltSinifNo"].map(str)

In [5]:
satisTarihIndex = satis.drop(['Tarih', 'Magaza', 'KategoriNo', 'KategoriGrupNo',
                              'KategoriAnaGrupNo', 'KategoriSinifNo', 'KategoriAltSinifNo'], axis = 1)
imhaTarihIndex = imha.drop(['Tarih', 'Magaza'], axis = 1)
stokTarihIndex = stok.drop(['Tarih', 'Magaza'], axis = 1)

In [6]:
mal['KategoriTamami'] = mal["KategoriAnaGrupNo"].map(
    str)+ "" +mal["KategoriGrupNo"].map(str)+ "" + mal["KategoriSinifNo"].map(
    str)+ "" + mal["KategoriAltSinifNo"].map(str)

In [7]:
mal['KategoriSinifTamami'] = mal["KategoriAnaGrupNo"].map(str)+ "" + mal["KategoriGrupNo"].map(
    str)+ "" + mal["KategoriSinifNo"].map(str)

In [8]:
malOzellik = mal.merge(mal[['MalNo','KategoriTamami']].groupby(
    'KategoriTamami').count().rename(columns = {'MalNo' : 'UrunSayisi'}),on = 'KategoriTamami', how = 'inner')

In [9]:
malOzellik['UrunSayisi'] = malOzellik['UrunSayisi'] -  1

In [10]:
malOzellik2 = malOzellik.merge(mal[['MalNo','KategoriSinifTamami']].groupby(
    'KategoriSinifTamami').count().rename(columns = {'MalNo' : 'UrunSinifSayisi'}),on = 'KategoriSinifTamami', 
                               how = 'inner')

In [11]:
malOzellik2['UrunSinifSayisi'] = malOzellik2['UrunSinifSayisi'] - 1

In [12]:
satis_stok = satisTarihIndex.merge(stokTarihIndex, how = 'outer', on = ['MalNo', 'TarihIndex'])

In [13]:
satis_stok_imha = satis_stok.merge(imhaTarihIndex, how = 'outer', on = ['MalNo', 'TarihIndex'])

In [14]:
total_data = satis_stok_imha.merge(malOzellik2, how = 'inner', on = 'MalNo')

In [15]:
#total_data.to_csv('raw_data-2016-satis-olan.csv', sep = ';', index = False)

In [16]:
total_data['Gun'] = total_data['TarihIndex'].dt.day
total_data['Ay'] = total_data['TarihIndex'].dt.month
total_data['Ceyrek'] = total_data['TarihIndex'].dt.quarter
total_data['Yil'] = total_data['TarihIndex'].dt.year
total_data['DayOfYear'] = total_data['TarihIndex'].dt.dayofyear
total_data['DayOfWeek'] = total_data['TarihIndex'].dt.dayofweek
total_data['MonthStart'] = total_data['TarihIndex'].dt.is_month_start
total_data['Weekend'] = np.where(total_data['DayOfWeek'] > 5,1,0)
total_data['MonthStart'] = np.where(total_data['MonthStart'] == True,1,0)

In [17]:
#total_data.to_csv('train_data_4.csv', sep = ';', index = False)

In [18]:
#total_data[total_data['MalNo'] == 12284590].sort_values('TarihIndex')
total_data_fillna = total_data.fillna(0)
#total_data_fillna['SatisNetTutar'] / total_data_fillna['SatisMiktar']

In [19]:
#total_data_fillna.to_csv('train_data_5.csv', sep = ';', index = False)

In [20]:
total_data_fillna['YilBasinaUzaklik'] = (pd.to_datetime(total_data_fillna['Yil'].map(str) + '-12-31') - total_data_fillna['TarihIndex'])/ np.timedelta64(1, 'D')

In [21]:
GunlukSatisOrtalamasi = total_data_fillna[['MalNo', 'SatisMiktar', 'DayOfWeek']].groupby(
    ['MalNo','DayOfWeek']).mean().rename(
    columns = {'SatisMiktar': 'GunlukSatisOrtalamasi'})

In [22]:
GunlukSatisMedian = total_data_fillna[['MalNo', 'SatisMiktar', 'DayOfWeek']].groupby(
    ['MalNo','DayOfWeek']).median().rename(
    columns = {'SatisMiktar': 'GunlukSatisMedian'})

In [23]:
satis['DayOfWeek'] = satis['TarihIndex'].dt.dayofweek
satis['Quarter'] = satis['TarihIndex'].dt.quarter

In [24]:
GunlukSatisOrt2016 = satis[satis['TarihIndex'] >= '2016-01-01'][['MalNo', 'DayOfWeek', 'SatisMiktar']].groupby(
    ['MalNo','DayOfWeek']).mean().rename(columns = {'SatisMiktar' : 'GunlukSatisOrt2016'})

In [25]:
GunlukSatisMed2016 = satis[satis['TarihIndex'] >= '2016-01-01'][['MalNo', 'DayOfWeek', 'SatisMiktar']].groupby(
    ['MalNo','DayOfWeek']).median().rename(columns = {'SatisMiktar' : 'GunlukSatisMed2016'})

GunlukSatisOrt2016q = satis[satis['TarihIndex'] >= '2016-01-01'][['MalNo', 'DayOfWeek',
                                                                  'Quarter', 'SatisMiktar']].groupby(
    ['MalNo','DayOfWeek', 'Quarter']).mean().rename(columns = {'SatisMiktar' : 'GunlukSatisOrt2016q'})

GunlukSatisMed2016q = satis[satis['TarihIndex'] >= '2016-01-01'][['MalNo', 'DayOfWeek',
                                                                  'Quarter', 'SatisMiktar']].groupby(
    ['MalNo','DayOfWeek', 'Quarter']).mean().rename(columns = {'SatisMiktar' : 'GunlukSatisOrt2016q'})

In [26]:
total_data_fillna = total_data_fillna.merge(GunlukSatisOrtalamasi, how = 'inner', on = ['MalNo', 'DayOfWeek'])

In [27]:
total_data_fillna = total_data_fillna.merge(GunlukSatisMedian, how = 'inner', on = ['MalNo', 'DayOfWeek'])

In [28]:
total_data_fillna = total_data_fillna.merge(GunlukSatisOrt2016, how = 'inner', on = ['MalNo', 'DayOfWeek'])

In [29]:
total_data_fillna = total_data_fillna.merge(GunlukSatisMed2016, how = 'inner', on = ['MalNo', 'DayOfWeek'])

In [30]:
total_data_fillna.head()

,MalNo,SatisMiktar,SatisNetTutar,TarihIndex,StokMiktar,ImhaMiktar,KategoriAnaGrupNo,KategoriGrupNo,KategoriSinifNo,KategoriAltSinifNo,...,Yil,DayOfYear,DayOfWeek,MonthStart,Weekend,YilBasinaUzaklik,GunlukSatisOrtalamasi,GunlukSatisMedian,GunlukSatisOrt2016,GunlukSatisMed2016
0,12284590,2.0,35.59,2014-01-01,1.0,0.0,300,10,11,10,...,2014,1,2,1,0,364.0,0.25,0.0,1.0,1.0
1,12284590,2.0,62.55,2014-12-24,2.0,0.0,300,10,11,10,...,2014,358,2,0,0,7.0,0.25,0.0,1.0,1.0
2,12284590,3.0,101.43,2015-12-09,14.0,0.0,300,10,11,10,...,2015,343,2,0,0,22.0,0.25,0.0,1.0,1.0
3,12284590,1.0,33.81,2016-12-21,4.0,0.0,300,10,11,10,...,2016,356,2,0,0,10.0,0.25,0.0,1.0,1.0
4,12284590,1.0,38.05,2016-12-28,3.0,0.0,300,10,11,10,...,2016,363,2,0,0,3.0,0.25,0.0,1.0,1.0


In [31]:
total_data_fillna.corr()

,MalNo,SatisMiktar,SatisNetTutar,StokMiktar,ImhaMiktar,KategoriAnaGrupNo,KategoriGrupNo,KategoriSinifNo,KategoriAltSinifNo,UrunSayisi,...,Yil,DayOfYear,DayOfWeek,MonthStart,Weekend,YilBasinaUzaklik,GunlukSatisOrtalamasi,GunlukSatisMedian,GunlukSatisOrt2016,GunlukSatisMed2016
MalNo,1.000000,-0.054722,-0.050967,0.019472,0.002571,0.044785,-0.129516,0.012412,-0.276335,0.078187,...,0.059914,0.030955,-0.000747,-0.000533,0.002993,-0.030691,-0.074168,-0.086240,-0.049374,-0.051418
SatisMiktar,-0.054722,1.000000,0.782576,0.116461,0.318204,-0.138769,-0.087822,0.142379,0.035451,-0.077349,...,0.020843,0.002193,0.043662,0.001088,0.046935,-0.002120,0.737811,0.671134,0.693686,0.673943
SatisNetTutar,-0.050967,0.782576,1.000000,-0.011462,0.234406,-0.148602,-0.054516,0.137211,-0.014262,-0.072290,...,0.021668,-0.017045,0.045822,0.002235,0.045266,0.017107,0.642388,0.622224,0.585337,0.555295
StokMiktar,0.019472,0.116461,-0.011462,1.000000,0.177305,-0.017822,-0.109608,0.012889,-0.017304,-0.000675,...,0.022234,0.005598,0.000149,0.003350,-0.005193,-0.005490,0.049079,-0.000952,0.077489,0.076154
ImhaMiktar,0.002571,0.318204,0.234406,0.177305,1.000000,-0.076511,-0.109709,0.000892,-0.009664,-0.016222,...,0.016803,0.017739,-0.001430,0.002070,0.001250,-0.017715,0.281420,0.277925,0.278536,0.282413
KategoriAnaGrupNo,0.044785,-0.138769,-0.148602,-0.017822,-0.076511,1.000000,0.469241,0.126273,0.018747,-0.165679,...,-0.033365,0.000961,0.001757,-0.006356,-0.002809,-0.001057,-0.188083,-0.181569,-0.178886,-0.172546
KategoriGrupNo,-0.129516,-0.087822,-0.054516,-0.109608,-0.109709,0.469241,1.000000,-0.048134,0.130523,-0.366196,...,-0.022565,-0.020887,0.000194,-0.003069,-0.001150,0.020764,-0.119030,-0.084415,-0.136507,-0.131274
KategoriSinifNo,0.012412,0.142379,0.137211,0.012889,0.000892,0.126273,-0.048134,1.000000,0.385416,-0.334530,...,-0.045144,-0.002372,-0.008646,-0.000607,-0.007475,0.002220,0.192976,0.183126,0.171069,0.159842
KategoriAltSinifNo,-0.276335,0.035451,-0.014262,-0.017304,-0.009664,0.018747,0.130523,0.385416,1.000000,-0.401033,...,-0.051082,-0.015791,-0.006994,-0.000333,-0.005026,0.015593,0.048049,0.061972,0.042335,0.054972
UrunSayisi,0.078187,-0.077349,-0.072290,-0.000675,-0.016222,-0.165679,-0.366196,-0.334530,-0.401033,1.000000,...,0.066985,0.009500,0.012265,0.000849,0.007728,-0.009211,-0.104836,-0.097689,-0.105303,-0.104176


In [32]:
total_data_fillna.to_csv('train_data_7.csv', sep = ';', index = False)